In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("review_and_category_analytics") \
    .config("spark.executor.memory", '8g') \
    .config('spark.executor.cores', '4') \
    .config('spark.cores.max', '4') \
    .config("spark.driver.memory",'8g') \
    .getOrCreate()

sc = spark.sparkContext

In [103]:
data = spark.read.csv('/project/ds5559/group14_tennis/tennis_atp/atp_matches_2012.csv', header = True)
data_path = '/project/ds5559/group14_tennis/tennis_atp/'

In [26]:
import os

# assign path
path, dirs, files = next(os.walk(data_path))
file_count = len(files)
# create empty list
dataframes_list = []
 
# append datasets to the list
for i in range(file_count):
    if '.csv' in files[i]:
        temp_df =  spark.read.csv(data_path+files[i])
        dataframes_list.append(temp_df)
     
# display datasets
'''for dataset in dataframes_list:
    dataset.show(5)
    print('\n')
'''

"for dataset in dataframes_list:\n    dataset.show(5)\n    print('\n')\n"

In [12]:
dataframes_list[5].show(5)

+--------------------+-----------------+-------+---------+-------------+------------+---------+---------+-----------+------------+-------------------+-----------+---------+----------+-------------+--------+----------+-----------+--------------------+----------+--------+---------+-------------+-----------+-------+-----+-------+-----+----+------+-------+--------+--------+-------+---------+---------+-----+----+------+-------+--------+--------+-------+---------+---------+-----------+------------------+----------+-----------------+
|                 _c0|              _c1|    _c2|      _c3|          _c4|         _c5|      _c6|      _c7|        _c8|         _c9|               _c10|       _c11|     _c12|      _c13|         _c14|    _c15|      _c16|       _c17|                _c18|      _c19|    _c20|     _c21|         _c22|       _c23|   _c24| _c25|   _c26| _c27|_c28|  _c29|   _c30|    _c31|    _c32|   _c33|     _c34|     _c35| _c36|_c37|  _c38|   _c39|    _c40|    _c41|   _c42|     _c43|     

In [10]:
!pwd

/sfs/qumulo/qhome/kgb3mf


In [11]:
first_dataset = dataframes_list[0]
first_dataset.show(5)

+----------+--------------+-------+---------+-------------+------------+---------+----------+----------+-----------+------------+---------+---------+----------+-----------+-------------+-------+-----+-------------------+------------+----------+-----------+-------------+----------------+------------+----------+-----------+-------------+----------------+-----------+---------+----------+-------------+--------------+-----------+---------+----------+-------------+------------+-------------------+------------+-------------------+-----------+------------------+-----------+------------------+-------+-----+----+------+-------+--------+--------+-------+---------+---------+-----+----+------+-------+--------+--------+-------+---------+---------+
|       _c0|           _c1|    _c2|      _c3|          _c4|         _c5|      _c6|       _c7|       _c8|        _c9|        _c10|     _c11|     _c12|      _c13|       _c14|         _c15|   _c16| _c17|               _c18|        _c19|      _c20|       _c21|

In [17]:
data.take(1)

[Row(tourney_id='2012-339', tourney_name='Brisbane', surface='Hard', draw_size='32', tourney_level='A', tourney_date='20120101', match_num='1', winner_id='104918', winner_seed='1', winner_entry=None, winner_name='Andy Murray', winner_hand='R', winner_ht='190', winner_ioc='GBR', winner_age='24.6351813826', loser_id='105062', loser_seed=None, loser_entry=None, loser_name='Mikhail Kukushkin', loser_hand='R', loser_ht='183', loser_ioc='KAZ', loser_age='24.0191649555', score='5-7 6-3 6-2', best_of='3', round='R32', minutes='130', w_ace='10', w_df='0', w_svpt='92', w_1stIn='57', w_1stWon='41', w_2ndWon='17', w_SvGms='15', w_bpSaved='4', w_bpFaced='7', l_ace='4', l_df='3', l_svpt='77', l_1stIn='41', l_1stWon='29', l_2ndWon='15', l_SvGms='14', l_bpSaved='4', l_bpFaced='9', winner_rank='4', winner_rank_points='7380', loser_rank='91', loser_rank_points='598')]

In [87]:
# CHange this to full dataset later
print("Number of columns:", len(data.columns))

Number of columns: 49


In [104]:
# Make a column called numNull then filter out if over 6
data = data.withColumn("numNull", sum(data[col].isNull().cast('int') for col in data.columns))

data = data.filter(data.numNull < 6)
data.count()

2676

In [105]:
data.show()

+----------+------------+-------+---------+-------------+------------+---------+---------+-----------+------------+-------------------+-----------+---------+----------+-------------+--------+----------+-----------+-------------------+----------+--------+---------+-------------+-----------------+-------+-----+-------+-----+----+------+-------+--------+--------+-------+---------+---------+-----+----+------+-------+--------+--------+-------+---------+---------+-----------+------------------+----------+-----------------+-------+
|tourney_id|tourney_name|surface|draw_size|tourney_level|tourney_date|match_num|winner_id|winner_seed|winner_entry|        winner_name|winner_hand|winner_ht|winner_ioc|   winner_age|loser_id|loser_seed|loser_entry|         loser_name|loser_hand|loser_ht|loser_ioc|    loser_age|            score|best_of|round|minutes|w_ace|w_df|w_svpt|w_1stIn|w_1stWon|w_2ndWon|w_SvGms|w_bpSaved|w_bpFaced|l_ace|l_df|l_svpt|l_1stIn|l_1stWon|l_2ndWon|l_SvGms|l_bpSaved|l_bpFaced|winner_

In [106]:

winners = [col for col in data.columns if col.startswith('w_')]
winners


['w_ace',
 'w_df',
 'w_svpt',
 'w_1stIn',
 'w_1stWon',
 'w_2ndWon',
 'w_SvGms',
 'w_bpSaved',
 'w_bpFaced']

In [107]:
losers = [col for col in data.columns if col.startswith('l_')]
losers

['l_ace',
 'l_df',
 'l_svpt',
 'l_1stIn',
 'l_1stWon',
 'l_2ndWon',
 'l_SvGms',
 'l_bpSaved',
 'l_bpFaced']

In [108]:

final_df = data.select(winners)
new_columns = [col.strip('w_') for col in winners]

for i in range(0, len(final_df.columns)):
    final_df = final_df.withColumnRenamed(final_df.columns[i], new_columns[i])


In [109]:
from pyspark.sql.functions import lit

final_df = final_df.withColumn('result', lit(1))
final_df.show(5)

+---+---+----+-----+------+------+-----+-------+-------+------+
|ace| df|svpt|1stIn|1stWon|2ndWon|SvGms|bpSaved|bpFaced|result|
+---+---+----+-----+------+------+-----+-------+-------+------+
| 10|  0|  92|   57|    41|    17|   15|      4|      7|     1|
| 23|  1| 104|   55|    47|    31|   17|      3|      3|     1|
|  8|  3|  71|   31|    26|    26|   11|      5|      6|     1|
|  1|  2|  75|   49|    31|    16|   13|      3|      6|     1|
| 10|  1|  56|   28|    23|    18|    8|      2|      2|     1|
+---+---+----+-----+------+------+-----+-------+-------+------+
only showing top 5 rows



In [110]:
final_df2 = data.select(losers)

for i in range(0, len(final_df2.columns)):
    final_df2 = final_df2.withColumnRenamed(final_df2.columns[i], new_columns[i])
    
final_df2 = final_df2.withColumn('result', lit(0))
final_df2.show(5)


+---+---+----+-----+------+------+-----+-------+-------+------+
|ace| df|svpt|1stIn|1stWon|2ndWon|SvGms|bpSaved|bpFaced|result|
+---+---+----+-----+------+------+-----+-------+-------+------+
|  4|  3|  77|   41|    29|    15|   14|      4|      9|     0|
|  3|  5| 103|   67|    54|    20|   16|      1|      2|     0|
| 14|  1|  79|   48|    36|    17|   11|      2|      4|     0|
|  4|  2|  90|   49|    27|    23|   13|      9|     14|     0|
|  2|  1|  55|   38|    28|    11|    9|      0|      0|     0|
+---+---+----+-----+------+------+-----+-------+-------+------+
only showing top 5 rows



In [111]:
final = final_df.union(final_df2)

final.show(5)

+---+---+----+-----+------+------+-----+-------+-------+------+
|ace| df|svpt|1stIn|1stWon|2ndWon|SvGms|bpSaved|bpFaced|result|
+---+---+----+-----+------+------+-----+-------+-------+------+
| 10|  0|  92|   57|    41|    17|   15|      4|      7|     1|
| 23|  1| 104|   55|    47|    31|   17|      3|      3|     1|
|  8|  3|  71|   31|    26|    26|   11|      5|      6|     1|
|  1|  2|  75|   49|    31|    16|   13|      3|      6|     1|
| 10|  1|  56|   28|    23|    18|    8|      2|      2|     1|
+---+---+----+-----+------+------+-----+-------+-------+------+
only showing top 5 rows



### Number of records

In [112]:
print('Number of records: ', final.count())

Number of records:  5352


### Number of columns

In [113]:
print('Number of columns: ', len(final.columns))

Number of columns:  10


### Statistical summary of response variable

In [114]:
print(final.toPandas().result.value_counts())

final.toPandas().describe()

# This is a binary response variable, so the only thing that really matters is value counts

1    2676
0    2676
Name: result, dtype: int64


,result
count,5352.000000
mean,0.500000
std,0.500047
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [115]:
# final.groupBy('result').count().orderBy('result')

### Statistical summary of potential predictor variables (if there are a large number of predictors, select the top 10)


In [118]:
final.toPandas()

,ace,df,svpt,1stIn,1stWon,2ndWon,SvGms,bpSaved,bpFaced,result
0,10,0,92,57,41,17,15,4,7,1
1,23,1,104,55,47,31,17,3,3,1
2,8,3,71,31,26,26,11,5,6,1
3,1,2,75,49,31,16,13,3,6,1
4,10,1,56,28,23,18,8,2,2,1
...,...,...,...,...,...,...,...,...,...,...
5347,12,1,88,55,43,20,15,0,1,0
5348,6,5,83,42,28,12,13,3,10,0
5349,5,2,79,48,32,15,13,6,10,0
5350,5,1,66,31,25,15,10,0,3,0


In [117]:
final.toPandas().describe()

,result
count,5352.000000
mean,0.500000
std,0.500047
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000
